<a href="https://colab.research.google.com/github/AdarshKhatri01/DBMS-Notes/blob/main/Transactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Transaction in DBMS**  
A **transaction** is a sequence of one or more database operations (like `INSERT`, `UPDATE`, `DELETE`, `SELECT`) executed as a single unit of work. Transactions ensure that the database remains **consistent and reliable**, even in the event of system failures.  

#### **Example of a Transaction:**  
Suppose you are transferring ₹1000 from Account A to Account B:  
1. **Withdraw ₹1000 from Account A**  
   ```sql
   UPDATE accounts SET balance = balance - 1000 WHERE account_id = 'A';
   ```
2. **Deposit ₹1000 into Account B**  
   ```sql
   UPDATE accounts SET balance = balance + 1000 WHERE account_id = 'B';
   ```
3. **Commit the transaction** (if both steps are successful)  
   ```sql
   COMMIT;
   ```
4. **Rollback if an error occurs**  
   ```sql
   ROLLBACK;
   ```  
If one step fails (e.g., system crash after withdrawal but before deposit), the transaction is **rolled back** to prevent data inconsistency.

---

# **ACID Properties of Transactions**  
To maintain **database integrity and reliability**, transactions follow the **ACID properties**:  

1. **Atomicity (A)** – *All or nothing!*  
   - Either **all** operations in a transaction execute successfully, or **none** of them are applied.  
   - Example: If money is deducted from Account A but not credited to Account B due to a failure, the whole transaction is rolled back.

2. **Consistency (C)** – *Database remains valid before and after the transaction.*  
   - A transaction ensures that the database remains in a valid state, following all integrity constraints.  
   - Example: If a transaction deducts ₹1000 from A, it must **add ₹1000 to B**, ensuring no data is lost.

3. **Isolation (I)** – *Transactions do not interfere with each other.*  
   - If multiple transactions run at the same time, they should not affect each other’s execution.  
   - Example: If two users transfer money at the same time, one transaction should **not see** half-completed changes of the other.

4. **Durability (D)** – *Once committed, the changes are permanent.*  
   - Even if a system crash occurs after a transaction is committed, the changes **must not be lost**.  
   - Example: If ₹1000 is transferred successfully and the system crashes, the transfer should **not be reversed**.

---

# **Concurrency Control in DBMS**  
When multiple transactions run simultaneously, **concurrency control** ensures **correctness and consistency** of the database. Without it, problems like **dirty reads, lost updates, and uncommitted data issues** can occur.

#### **Common Concurrency Issues & Solutions**
| Issue | Description | Solution |
|--------|-----------------|------------|
| **Dirty Read** | Reading uncommitted changes from another transaction | Use **LOCKS** or **Isolation Levels** |
| **Lost Update** | Two transactions modify the same data, and one update gets lost | Use **LOCKS** or **Versioning** |
| **Phantom Read** | One transaction sees new rows added by another transaction | Use **Serializable Isolation** |

#### **Example of Concurrency Issue (Lost Update)**  
Two users update the same account balance at the same time:

1. **User 1 Reads Balance = ₹5000**
2. **User 2 Reads Balance = ₹5000**
3. **User 1 Updates Balance to ₹6000**
4. **User 2 Updates Balance to ₹5500** (Overwrites User 1’s update!)

🔴 **Solution:** Use **locking mechanisms** or **transactions with isolation levels** to prevent simultaneous updates.

---

#### **3. Concurrency Control in DBMS**
Concurrency control ensures that multiple transactions can execute simultaneously **without leading to inconsistency or data corruption**.

- **Problems in Concurrency Control:**
  1. **Lost Update Problem** – Two transactions update the same data, and one update is lost.
  2. **Dirty Read Problem** – A transaction reads uncommitted changes of another transaction.
  3. **Non-Repeatable Read** – A transaction reads the same data twice and gets different results.
  4. **Phantom Read** – A transaction reads a different number of rows in two reads due to another transaction inserting/deleting records.

- **Concurrency Control Techniques:**
  1. **Lock-based Protocols**:
     - **Shared Lock (S)** – Multiple transactions can read but not write.
     - **Exclusive Lock (X)** – Only one transaction can read and write.
  2. **Timestamp-based Protocols**:
     - Assigns a timestamp to each transaction to maintain order.
  3. **Optimistic Concurrency Control**:
     - Transactions proceed without restrictions and are validated before committing.
  4. **Multiversion Concurrency Control (MVCC)**:
     - Maintains multiple versions of data for different transactions.

---

### **Conclusion**  
- Transactions ensure **data consistency and integrity** using **ACID properties**.  
- **Concurrency control** prevents **data conflicts** when multiple transactions run together.  
- DBMS uses **locking, isolation levels, and versioning** to handle concurrency.  



# **Transaction States in DBMS**  

## **1. What is a Transaction?**  
A **transaction** is a **sequence of database operations** that must be executed as a **single unit**. It ensures **Atomicity, Consistency, Isolation, and Durability (ACID properties).**  

A transaction can be in different **states** during its execution, from start to commit or rollback.

---

## **2. Transaction States with Diagram**
A transaction in DBMS **moves through different states** before being committed or aborted.

### **Transaction States Diagram**
```plaintext
        +-----------+       +------------+
        |   Active  | ----> | Partially  |  
        |           |       | Committed  |  
        +-----------+       +------------+
              |                     |
              |                     v
              |              +--------------+
              |              |   Committed  |
              |              +--------------+
              v
       +-----------+       +------------+
       |   Failed  | ----> |   Aborted   |
       +-----------+       +------------+
```

---

### **3. Explanation of Transaction States**  

### **1️⃣ Active State**  
- The transaction **starts** and is executing operations (INSERT, UPDATE, DELETE, etc.).
- If any failure occurs, the transaction moves to the **Failed state**.

✅ **Example:**  
```sql
START TRANSACTION;
UPDATE Account SET Balance = Balance - 500 WHERE Account_ID = 101;
```
🔹 The transaction is still active as it hasn't been committed.

---

### **2️⃣ Partially Committed State**  
- The transaction has **executed all operations**, but changes are not yet **permanent**.  
- It enters this state **just before commit**.

✅ **Example:**  
```sql
UPDATE Account SET Balance = Balance + 500 WHERE Account_ID = 102;
```
🔹 The second update is done, but the transaction is **not yet committed**.

---

### **3️⃣ Committed State**  
- The transaction **is successfully completed**.  
- Changes are **permanently saved in the database**.  
- The system ensures **Durability (ACID property)**.

✅ **Example:**  
```sql
COMMIT;
```
🔹 The transaction moves to the **Committed state** after commit.

---

### **4️⃣ Failed State**  
- If the transaction **encounters an error** (e.g., system crash, constraint violation), it enters the **Failed state**.  
- Changes made by the transaction are **invalid** and should not be saved.

❌ **Example:**  
- A system crash occurs before commit.
- A constraint violation happens (e.g., inserting duplicate PRIMARY KEY).

---

### **5️⃣ Aborted State**  
- If a transaction **fails**, it must be **rolled back** (undo changes).  
- The transaction is either restarted or permanently terminated.

✅ **Example:**  
```sql
ROLLBACK;
```
🔹 All changes made by the transaction are **undone**, and the database remains **unchanged**.

---

## **4. Summary of Transaction States**
| **State** | **Description** |
|-----------|----------------|
| **Active** | Transaction is in progress. |
| **Partially Committed** | All operations are executed, but changes are not saved yet. |
| **Committed** | Changes are permanently saved. |
| **Failed** | An error or system failure occurs. |
| **Aborted** | Changes are rolled back, and the transaction is terminated. |

---

## **5. Conclusion**
Transactions ensure **data consistency and reliability** in DBMS. The **state diagram** helps understand how a transaction **progresses from execution to commit or rollback**.